<a href="https://colab.research.google.com/github/Ian326/TI3001C/blob/main/E5_Construcci%C3%B3n_y_transformaci%C3%B3n_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recolección y descripción de datos
---
***Equipo:***

**Ruth Jiménez Vázquez - A01351923**

**José Ignacio Gómez Moreno - A01067362**

**Harry Hernández Grande - A01736341**

**Ian Joab Padrón Corona - A01708940**

---



## Carga de datos

In [ ]:
'''
===============================================================================
Librerias de Python a utilizar en el proyecto
===============================================================================
pandas: Libreria de manipulacion de datos en DataFrames
numpy: Libreria de manipulacion de datos en arrays o funciones matematicas
missingno: Libreria de visualizacion de datos nulos
CountEncoder: Convertir variables categoricas en numericas asignandoles un valor de acuerdo a su relevancia
OneHotEncoder: Convertir variables categoricas en numericas creando columnas binarias
matplotlib: Libreria de visualizacion de datos a traves de graficos
seaborn: Libreria de visualizacion de datos a traves de graficos. (Extension de matplotlib)
stats: Libreria de estadisticas para realizar pruebas de normalidad
PowerTransformer: Transformar variables numericas a traves de una transformacion de potencia. Para normalizar los datos
LinearRegression: Modelo de regresion lineal
LogisticRegression: Modelo de regresion logistica
sm: Libreria de modelos estadisticos
'''

import pandas as pd
import numpy as np

import missingno as msno

from category_encoders import CountEncoder
from category_encoders import OneHotEncoder

import matplotlib.pyplot as plt

import seaborn as sns

import scipy.stats as stats

from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_curve

import statsmodels.api as sm

In [ ]:
print(pd.read_csv('../content/vista_repdetail_tec.csv', low_memory=False).info())
# Como se puede observar, el archivo .CSV no contiene el nombre de las columnas a utilizar.
# Debido a esto, utiliza el primer registro como nombre de las columnas.
# Por tanto, se debera asignar manualmente el nombre de las columnas según el diccionario de datos 
# proporcionado por el cliente.

In [ ]:
# Nombres de las columnas según el Diccionario de Datos proporcionado
columns = [
    "Column", "Taller", "OrderID", "kMS", "OpenedDate", "ClosedDate", "Status",
    "lagdias", "laghoras", "ClosedDay", "ClosedMonth", "ClosedYear", "OpenedDay",
    "OpenedMonth", "OpenedYear", "CreatedBy", "ClosedBy", "RepReason",
    "OrderType", "Supplier", "COMPCDKEY", "COMPCODE", "DESCRIP", "QTYRCVD",
    "QTYREQD", "CHGAMT", "TAXAMT", "SUBTOTAL", "TOTAL", "UnitID", "UnitType",
    "UnitYear", "EC", "Flota", "ReasonType", "COMPLAINT", "Jobcode", "TipoLinea",
    "NumParte", "FechaPromesa", "Estimate", "CompleteDay", "CompleteMonth",
    "CompleteYear", "CompleteDate", "MECHANIC", "CITA"
]

# Cargar el archivo .CSV con los nombres de las columnas asignados a un DataFrame 'data'
data = pd.read_csv('../content/vista_repdetail_tec.csv', names=columns, low_memory=False)
# Mostrar la informacion del DataFrame creado
data.info()

In [ ]:
# Visualizacion de una muestra de los datos en el DataFrame
data.sample(3)

## Documentación de los esquemas de datos

* ¿Cuál es nuestra fuente de datos?

Actualmente, la información es recabada por nuestro SocioFormador TDR. Tenemos acceso a un escritorio remoto que está a su vez conectado con un servidor de Microsoft SQL Server.

A partir de dicho servidor, llamamos a una de las vistas a las que nos dieron acceso, para recabar toda la información sobre las reparaciones de su flota.


* ¿Cómo obtenemos la información?

A traves de una Query de SQL a la que nos autorizaron, que seria la siguiente:


```
SELECT * FROM [TMWSuite].[dbo].[vista_repdetail_tec]
```




## Diccionario de datos

| Column        | Descripción                                                                                       |
| ------------- | ------------------------------------------------------------------------------------------------- |
| OrderID       | ID de la orden de reparación                                                                      |
| Taller        | Acrónimo del taller donde se realizó la reparación                                                |
| kMS         | Kilometros viajados por la unidad                                                                     |
| OpenedDate    | Fecha de apertura de la orden de reparación                                                       |
| ClosedDate    | Fecha de cierre de la orden de reparación                                                         |
| Status        | Estatus de la orden de reparación                                                                 |
| lagdias       | Días entre cierre y apertura de la orden de reparación                                            |
| laghoras      | Horas entre cierre y apertura de la orden de reparación                                           |
| ClosedDay     | Día del mes en que se cerró la orden de reparación                                                |
| ClosedMonth   | Mes que se cerró la orden de reparación                                                           |
| ClosedYear    | Año en que se cerró la orden de reparación                                                        |
| OpenedDay     | Día del mes en que se abrió la orden de reparación                                                |
| OpenedMonth   | Mes que se abrió la orden de reparación                                                           |
| OpenedYear    | Año en que se abrió la orden de reparación                                                        |
| CreatedBy     | Usuario que creó la orden de reparación                                                           |
| ClosedBy      | Usuario que cerro la orden de reparación                                                          |
| RepReason     | Razón de reparación                                                                               |
| OrderType     | Tipo de orden. Standar es en uno de nuestros talleres propios. Vendor es con un proveedor externo |
| Supplier      | Proveedor que realizó la reparación                                                               |
| COMPCDKEY     | Codigo de componente (No aporta info importante)                                                  |
| COMPCODE      | Codigo utilizado para identificar el tipo de reparación                                           |
| DESCRIP       | Descripción de la reparación                                                                      |
| QTYRCVD       | Cantidad de piezas o trabajo recibidas                                                            |
| QTYREQD       | Cantidad de piezas o trabajo requeridas                                                           |
| CHGAMT        | Costo unitario                                                                                    |
| TAXAMT        | IVA                                                                                               |
| SUBTOTAL      | Subtotal de la reparación (No incluye IVA)                                                        |
| TOTAL         | Total de la reparación (Incluye IVA)                                                              |
| UnitID        | Numero de unidad                                                                                  |
| UnitType      | Tipo de unidad (Tracto, Caja Seca, Dolly, etc.)                                                   |
| UnitYear      | Año del modelo de la unidad                                                                       |
| EC            | Equipo administrativo que se encarga de la operación de un conjunto de proyectos                  |
| Flota         | Nombre del proyecto en que trabaja el tracto. (Cliente)                                           |
| ReasonType    | Razón (No aporta info importante)                                                                 |
| COMPLAINT     | Queja reportada                                                                                   |
| Jobcode       | Numero para identificar el tipo de reparación                                                     |
| TipoLinea     | Describe el tipo de línea en la orden de reparación (Piezas, mano de obra, servicios, etc.)       |
| NumParte      | Numero de parte descarga en la línea de reparación                                                |
| FechaPromesa  | Fecha de promesa para terminar las reparaciones de la unidad                                      |
| Estimate      | Monto estimado (No lo utilizamos)                                                                 |
| CompleteDay   | Día del mes en que se completó la orden                                                           |
| CompleteMonth | Mes que se completó la orden de reparación                                                        |
| CompleteYear  | Año en que se completó la orden de reparación                                                     |
| CompleteDate  | Fecha en que se completó la orden de reparación                                                   |
| MECHANIC      | Mecánico interno que realizó la reparación                                                        |
| CITA          | Indica si la unidad tenía una cita agenda previamente                                             |

## Análisis de la estructura de los datos

* ¿Que tipo de datos tenemos? **Tenemos 13 de tipo float, 4 int, y 30 object.**
* ¿Requerimos modificar el tipo de datos de alguna columna? **Sí, para ser precisos, debemos convertir las siguientes columnas:**
  * **A int64**:
    * lagdias: Son cantidad de dias, no pueden venir con decimales
    * laghoras: Son cantidad de horas, no pueden venir con decimales
    * ClosedDay: Es el numero del dia en que fue cerrada (puede ser de 1 - 31)
    * QTYRCVD: Es una cantidad de piezas, no puede venir con decimales
    * QTYREQD: Es una cantidad de piezas, no puede venir con decimales
    * CompleteDay: Es el numero del dia en que fue cerrada (puede ser de 1 - 31)
    * CompleteYear: Es un año (no puede tener punto decimal o ser un string)
  
  * **A datetime64**
    * OpenedDate: Actualmente es un object, debe ser fecha
    * ClosedDate: Actualmente es un object, debe ser fecha
    * FechaPromesa: Actualmente es un object, debe ser fecha
    * CompleteDate: Actualmente es un object, debe ser fecha

* ¿Existen nulos? **Sí, actualmente hay 689,643 registros faltantes en toda la informacion que tenemos**

In [ ]:
# Conteo de valores nulos en el DataFrame
data.isnull().sum()

In [ ]:
print(data.isnull().sum().sum()) # <- Cantidad de nulos

## Perfiles involucrados

*   Equipo de TDR que nos puede ayudar con los datos (Identificar por nombre y cargo)


### Ing. Emmanuel Vargas Pérez

**Jefe de Administración de Equipo**

Nos proporcionará información sobre el proyecto que desarrollaremos (Sayer), incluyendo configuraciones de las flotas de tractocamiones y las fallas más comunes que suelen presentar estos vehículos.






### Ing. Mauricio Romo Ávila

**Subdirector de Mantenimiento**

Nos brindará información sobre el mantenimiento de los tractocamiones, incluyendo las fallas comunes, los costos asociados a estas fallas, las ubicaciones donde ocurren con mayor frecuencia, el número de talleres disponibles (tanto propios como generales), y responderá dudas sobre sus mantenimientos correctivos.


### Ing. Brandon Velazco

**Jefe de Taller**

Aportará información relevante sobre los tractocamiones, incluyendo detalles sobre las partes y componentes específicos que suelen fallar, en qué consisten estas fallas, así como la duración y el costo de las mismas.

*   Los profesores nos brindarán las herramientas y conocimientos
necesarios para lograr las metas y objetivos.

## Limpieza de Indices y Columnas

* ¿Qué indices vamos a utilizar?
 **La mejor opcion seria dejar un valor numérico discreto como índice, con el fin de que no tenga valores repetidos**

* ¿Qué columnas requieren un cambio de Tipo de Dato?
  * **A int64**:
      * lagdias: Son cantidad de días (enteros), no pueden venir con decimales
      * laghoras: Son cantidad de horas (enteros), no pueden venir con decimales
      * ClosedDay: Es el número del día en que fue cerrada (puede ser de 1 - 31)
      * QTYRCVD: Es una cantidad de piezas (enteras), no puede venir con decimales
      * QTYREQD: Es una cantidad de piezas (enteras), no puede venir con decimales
      * CompleteDay: Es el número del día en que fue cerrada (puede ser de 1 - 31)
      * CompleteYear: Es un año (no puede tener punto decimal o ser un string)

  * **A float64**:
    * Estimate: Se encuentra como object actualmente, debe ser un precio en decimal
  
  * **A datetime64**
    * OpenedDate: Actualmente es un object, debe ser fecha
    * ClosedDate: Actualmente es un object, debe ser fecha
    * FechaPromesa: Actualmente es un object, debe ser fecha
    * CompleteDate: Actualmente es un object, debe ser fecha


In [ ]:
# Convertir las columnas requeridas a 'DateTime'
data['OpenedDate'] = pd.to_datetime(data['OpenedDate'])
data['ClosedDate'] = pd.to_datetime(data['ClosedDate'])
data['FechaPromesa'] = pd.to_datetime(data['FechaPromesa'])
data['CompleteDate'] = pd.to_datetime(data['CompleteDate'])

In [ ]:
# Redondear los decimales de las columnas requeridas a 0, para posteriormente convertirlas a 'Int'
data['lagdias'] = data['lagdias'].round(0)
data['laghoras'] = data['laghoras'].round(0)
data['ClosedDay'] = data['ClosedDay'].round(0)
data['ClosedYear'] = data['ClosedYear'].round(0)
data['QTYRCVD'] = data['QTYRCVD'].round(0)
data['QTYREQD'] = data['QTYREQD'].round(0)
data['CompleteDay'] = data['CompleteDay'].round(0)
data['CompleteYear'] = data['CompleteYear'].round(0)

In [ ]:
# Llenar los valores nulos de las columnas requeridas con 0, para posteriormente convertirlas a 'Int'
data['lagdias'] = data['lagdias'].fillna(0)
data['laghoras'] = data['laghoras'].fillna(0)
data['ClosedDay'] = data['ClosedDay'].fillna(0)
data['ClosedYear'] = data['ClosedYear'].fillna(0)
data['QTYRCVD'] = data['QTYRCVD'].fillna(0)
data['QTYREQD'] = data['QTYREQD'].fillna(0)
data['CompleteDay'] = data['CompleteDay'].fillna(0)
data['CompleteYear'] = data['CompleteYear'].fillna(0)

In [ ]:
# Convertir las columnas requeridas a 'Int'
data['lagdias'] = data['lagdias'].astype(int)
data['laghoras'] = data['laghoras'].astype(int)
data['ClosedDay'] = data['ClosedDay'].astype(int)
data['ClosedYear'] = data['ClosedYear'].astype(int)
data['QTYRCVD'] = data['QTYRCVD'].astype(int)
data['QTYREQD'] = data['QTYREQD'].astype(int)
data['CompleteDay'] = data['CompleteDay'].astype(int)
data['CompleteYear'] = data['CompleteYear'].astype(int)

In [ ]:
# Convertir columnas requeridas a 'Float'
# data['Estimate'] = data['Estimate'].astype(float)
# No se puede, porque la informacion en la columna 'Estimate' no es realmente un estimado

In [ ]:
# Volver a revisar el tipo de dato de las columnas
data.info()

## Análisis de los datos (Descriptiores Estadisticos)

In [ ]:
# Seleccionar sólo las variables relevantes para las estadisticas
data_descriptive = data[['lagdias', 'laghoras','TOTAL','UnitYear',
                      'RepReason', 'UnitType', 'TipoLinea', 'CITA', 'COMPLAINT']]

data_descriptive.sample(10)

**Estadística descriptiva de las variables numéricas**

In [ ]:
# Estadística descriptiva de las variables numéricas
data_descriptive.describe()

| Variable numérica | Descripción | Rango | Desviación estándar |
|-----------|-----------|-----------|-----------|
| lagdias    | Días entre cierre y apertura de la orden de reparación   | 655    | Datos no tan dispersos    |
| laghoras    | Horas entre cierre y apertura de la orden de reparación    | 15,720    | Datos muy dispersos    |
| TOTAL    | Total de la reparación (Incluye IVA)   | 1,086,544   | Datos demasiado dispersos   |
| UnitYear    | Año del modelo de la unidad   | 116   | Datos no dispersos   |




*   **lagdias**: Los días de apertura y cierre de una reparación están ligeramente dispersos
*   **laghoras**: Las horas de apertura y cierre de una reparación presentan un gran rango de diferencia entre el valor máximo y mínimo, además de que están muy dispersas.
*   **TOTAL**: El costo total de las reparaciones tienen el rango más amplio, y con una alta desviación estándar.
*   **UnitYear**: Al tratarse de los años del modelo de cada unidad, no se presenta tanta dispersión.





In [ ]:
msno.matrix(data_descriptive[['lagdias', 'laghoras', 'TOTAL', 'UnitYear']])
#No hay notoria presencia de valores null

In [ ]:
data_descriptive.boxplot(figsize = (16, 8))
#El costo total de una reparación presenta la mayor cantidad de outliers, valores extremos
#Es importante consultar si estos valores son correctos, o pudiera tratarse de un error, de lo contrario se deberían eliminarse o imputarse

#El resto de variables no presentan tantos valores extremos

In [ ]:
#Histograma para analizar la variable más dispersa, TOTAL

data_descriptive.hist(['TOTAL'], figsize = (16,8))

**Estadística descripctiva de variables categóricas**

In [ ]:
# Estadística descripctiva de variables categóricas
data_descriptive.describe(include=object)

In [ ]:
# Valores más repetidos en las variables cualitativas
data_categorical = data_descriptive.select_dtypes(include=object)
for column in data_categorical.columns:
  print(data_categorical[column].value_counts().head(5))
  print()
# Borrar las variables temporales de la memoria
del column

Identificar si hay columnas comunes en los dataframes empleados que permitan “enlazar” los registros en las siguientes fases.

**Debido a que solo utilizamos un dataframe para esta parte inicial del proyecto, no contamos con columnas por las que se pueda enlazar a otros dataframes. A futuro sería posible que tengamos más de uno que sí valdría la pena enlazar a traves de llaves o columnas en común.**

# **Selección y limpieza de datos**

## Eliminar las columnas no relevantes

Columna|Justificacion
------|--------------
Column| No sabemos que es
Taller| Viene implicito en el OrderID
ClosedDate| Tenemos una metrica mas precisa (laghoras)
ClosedDay| ""
ClosedMonth| ""
ClosedYear| ""
OpenedDay| Tenemos una metrica mas general (OpenedDate)
OpenedMonth| ""
OpenedYear| ""
CreatedBy| No afecta realmente quien inicio la orden de reparacion
ClosedBy| No afecta realmente quien cerro la orden de reparacion
OrderType| Solo existen 2 tipos 'Standard' & 'Vendor'. Asumimos que no tiene efecto en la reparacion
Supplier| No afecta realmente de donde se obtuvo la pieza de reparacion
**COMPCDKEY**| **No sabemos si tiene utilidad o no**
**COMPCODE**| **No sabemos si tiene utilidad o no**
DESCRIP| Tiene informacion demasiado especifica para cada caso, no nos seria util
QTYRCVD| No afecta realmente si se daño 1 o mas piezas, si no, que fallaron
QTYREQD| Similar al anterior.
CHGAMT| Lo importante es la metrica del costo por reparacion (TOTAL)
TAXAMT| ""
SUBTOTAL| ""
EC| No afecta realmente el equipo administrativo del proyecto
ReasonType| Segun el diccionario de datos, no aporta informacion relevante
FechaPromesa| No genera impacto en la reparacion
Estimate| " "
CompleteDay| Tenemos una metrica mas precisa (laghoras)
CompleteMonth| ""
CompleteYear| ""
CompleteDate| ""
MECHANIC| Asumimos que no genera impacto en la reparacion quien la hizo

In [ ]:
# Seleccionar sólo las columnas relevantes para las estadisticas
data_filtered_cols = data.drop(['Column', 'Taller', 'ClosedDate', 'Status',
                   'ClosedDay', 'ClosedMonth', 'ClosedYear', 'OpenedDay', 'OpenedMonth', 'OpenedYear',
                   'CreatedBy', 'ClosedBy', 'OrderType', 'Supplier', 'COMPCDKEY', 'COMPCODE',
                   'QTYRCVD', 'QTYREQD', 'CHGAMT', 'TAXAMT', 'SUBTOTAL', 'EC',
                   'ReasonType', 'FechaPromesa', 'Estimate', 'CompleteDay',
                   'CompleteMonth', 'CompleteYear', 'CompleteDate', 'MECHANIC'], axis=1)

# Visualizacion de una muestra de los datos en el DataFrame
data_filtered_cols.sample(10)

## Limpieza de las columnas (Trim)

Notamos que algunas de las columnas extraídas desde el archivo .CSV tenían espacios en blanco. Dichos espacios deben ser removidos para evitar complicaciones más adelante en la agrupación y filtrado de información. Es por eso que procedemos a arreglar dichos errores en la captura de la información

In [ ]:
# Visualizacion de la informacion del DataFrame
data_filtered_cols.info()

In [ ]:
# Eliminar todos los registros donde 'kMS' sea nulo o menor a 40000
# data_filtered_cols = data_filtered_cols.drop(data_filtered_cols[data_filtered_cols['kMS'].isnull()].index)
data_filtered_cols = data_filtered_cols.drop(data_filtered_cols[data_filtered_cols['kMS'] < 40000].index)

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'RepReason'
data_filtered_cols['RepReason'] = data_filtered_cols['RepReason'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['RepReason'].value_counts().index[0:5]

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'UnitType'
data_filtered_cols['UnitType'] = data_filtered_cols['UnitType'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['UnitType'].value_counts().index

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'TipoLinea'
data_filtered_cols['Flota'] = data_filtered_cols['Flota'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['Flota'].value_counts().index[0:5]

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'COMPLAINT'
data_filtered_cols['COMPLAINT'] = data_filtered_cols['COMPLAINT'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['COMPLAINT'].value_counts().index[0:5]

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'Jobcode'
data_filtered_cols['Jobcode'] = data_filtered_cols['Jobcode'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['Jobcode'].value_counts().index[0:5]

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'TipoLinea'
data_filtered_cols['TipoLinea'] = data_filtered_cols['TipoLinea'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['TipoLinea'].value_counts().index

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'NumParte'
data_filtered_cols['NumParte'] = data_filtered_cols['NumParte'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['NumParte'].value_counts().index[0:5]

In [ ]:
# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'CITA'
data_filtered_cols['CITA'] = data_filtered_cols['CITA'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['CITA'].value_counts().index

Para la ultima columna 'CITA' se observa que existen registros con un campo vacio, por lo que se procede a sustituir este campo N/A por un texto de referencia (DESCONOCIDO). Además podemos corregir los nombres de algunos tipos de cita: 

* 'CITA A TIEMP' -> 'CITA A TIEMPO'
* 'EXPRES' -> 'EXPRESS'
* 'SCHEDULED' -> 'CON CITA'

In [ ]:
# Sustituir valores de 'CITA' con un string vacio por 'Desconocido'
data_filtered_cols['CITA'] = data_filtered_cols['CITA'].replace('', 'DESCONOCIDO')

# Sustituir valores de 'CITA A TIEMP' por 'CITA A TIEMPO'
data_filtered_cols['CITA'] = data_filtered_cols['CITA'].replace('CITA A TIEMP', 'CITA A TIEMPO')

# Sustituir valores de 'EXPRES' por 'EXPRESS'
data_filtered_cols['CITA'] = data_filtered_cols['CITA'].replace('EXPRES', 'EXPRESS')

# Sustituir valores de 'SCHEDULED' por 'CON CITA'
data_filtered_cols['CITA'] = data_filtered_cols['CITA'].replace('SCHEDULED', 'CON CITA')

# Verificar que se haya hecho correctamente
data_filtered_cols['CITA'].value_counts().index

## Limpieza de columnas (DESC)

Para esta columna, necesitamos asegurarnos que todas las palabras vengan en mayusculas, sin espacios en blanco al inicio y al final y además aegurarnos que no haya valores faltantes

In [ ]:
# Modificar nombre de columna 'DESCRIP' por 'DESCRIPTION'
data_filtered_cols.rename(columns={'DESCRIP': 'DESCRIPTION'}, inplace=True)

# Eliminacion de espacios en blanco al inicio y final de los valores de la columna 'DESCRIPTION'
data_filtered_cols['DESCRIPTION'] = data_filtered_cols['DESCRIPTION'].str.strip()

# Convertir la columna 'DESCRIPTION' a mayusculas
data_filtered_cols['DESCRIPTION'] = data_filtered_cols['DESCRIPTION'].str.upper()

# Verificar que la columna no tenga valores nulos
print(f'Cantidad de valores nulos en la columna DESCRIPTION: {data_filtered_cols["DESCRIPTION"].isnull().sum()}')

## Restablecer los índices con identificadores adecuados.

El índice del dataframe ya está establecido, considerando evitar los valores duplicados.

## Seleccionar o filtrar los registros u observaciones de interés
Con la finalidad de operar dataframes de menor complejidad.

Dado el proyecto asignado al equipo, nos centraremos en la informacion sobre todas las reparaciones de la flota 'Sayer', por lo que deberemos filtrar dicha informacion.

In [ ]:
# Extraemos la informacion del proyecto Sayer Full
data_sayer = data_filtered_cols[data_filtered_cols['Flota'] == 'Sayer Full'][:]
# Borrar la columna 'Flota', pues ya no es necesaria
data_sayer = data_sayer.drop('Flota', axis=1)
data_sayer.info()

In [ ]:
# Eliminar dataFrames que ya no se utilizaran
del data_descriptive, data_categorical, data_filtered_cols, columns

## Identificar valores faltantes y eliminarlos

In [ ]:
# Visualización de datos nulos en el DataFrame
data_sayer.isnull().sum()

In [ ]:
# Se identifican valores nulos en las columnas 'kMS', 'TipoLinea', 'CITA' y 'NumParte'
data_sayer[['TipoLinea', 'CITA', 'NumParte']].isnull().sum()

In [ ]:
# Visualizacion de datos nulos en el DataFrame
data_sayer[data_sayer.isnull().any(axis=1)]

In [ ]:
# Sustituir valores nulos de la columna 'CITA' por 'DESCONOCIDO'
data_sayer['CITA'] = data_sayer['CITA'].fillna('DESCONOCIDO')
data_sayer['CITA'].isnull().sum()

In [ ]:
# Sustituir valores nulos de la columna 'TipoLinea' por 'DESCONOCIDO'
data_sayer['TipoLinea'] = data_sayer['TipoLinea'].fillna('DESCONOCIDO')
data_sayer['TipoLinea'].isnull().sum()

In [ ]:
# Sustituir valores nulos de la columna 'NumParte' por 'DESCONOCIDO'
data_sayer['NumParte'] = data_sayer['NumParte'].fillna('DESCONOCIDO')
data_sayer['NumParte'].isnull().sum()

In [ ]:
# Verificar que se hayan sustituido correctamente los valores nulos
data_sayer.isnull().sum()

## Detectar valores atípicos para su posterior transformación.


In [ ]:
data_sayer.boxplot(vert=False)
#Observamos que en la columna de TOTAL es la que aparentemente hay mas datos
#atipicos, por lo que vamos a anilizarla a detalle


In [ ]:
# Información estadística del DataFrame
data_sayer.describe()

In [ ]:
# Análisis de la columna 'TOTAL'
data_sayer[['TOTAL']].boxplot(vert=False)

In [ ]:
# Visualizacion de datos atipicos en la columna 'TOTAL'
data_sayer[data_sayer['TOTAL'] > 30000]

In [ ]:
# Máscaras para identificar valores atipicos en la columna 'TOTAL'

# Consideramos que no es congruente que la columna 'TOTAL' tenga valores negativos
sayer_total_atipicosBajos = data_sayer['TOTAL'] <= 20

# Concatenar las mascaras para obtener los valores atipicos
valores_atipicos = sayer_total_atipicosBajos

# Conteo de valores atipicos
valores_atipicos.sum()

In [ ]:
# Creación de un nuevo DataFrame sin los valores atipicos
data_sayer_clean = data_sayer[~valores_atipicos]

In [ ]:
# Análisis de la columna 'TOTAL' sin valores atipicos
data_sayer_clean[['TOTAL']].boxplot(vert=False)

In [ ]:
# Análisis de la columna 'laghoras'
data_sayer_clean[['laghoras']].boxplot(vert=False)

In [ ]:
# Visualizacion de datos atipicos en la columna 'laghoras'
data_sayer_clean[data_sayer_clean['laghoras'] < 0]

In [ ]:
# Máscaras para identificar valores atipicos en la columna 'laghoras'

# Consideramos que no es congruente que la columna 'laghoras' tenga valores negativos o en 0
sayer_laghoras_atipicosBajos = data_sayer_clean['laghoras'] <= 0

# Consideramos que es posible que una reparacion sea bastante tardada. Es por ello que
# consideramos mantener los tiempos elevados, exceptuando aquellos que están muy alejados
# que podemos calcular en tiempos superiores a 1500 horas (62.5 dias)
sayer_laghoras_atipicosAltos = data_sayer_clean['laghoras'] >= 1500

# Concatenar las mascaras para obtener los valores atipicos
valores_atipicos = sayer_laghoras_atipicosBajos | sayer_laghoras_atipicosAltos

# Conteo de valores atipicos
valores_atipicos.sum()

In [ ]:
# Actualizacion del DataFrame sin los valores atipicos
data_sayer_clean = data_sayer_clean[~valores_atipicos]

In [ ]:
# Análisis de la columna 'laghoras' sin valores atipicos
data_sayer_clean[['laghoras']].boxplot(vert=False)

In [ ]:
# Análisis de la columna 'lagdias'. Como podemos observar, al imputar la columna 'laghoras'
# también se imputa la columna 'lagdias' ya que ambas eran un cálculo de la diferencia entre las fechas
data_sayer_clean[['lagdias']].boxplot(vert=False)

In [ ]:
# Verificar que no haya valores nulos en el DataFrame
data_sayer_clean.info()

In [ ]:
# Análisis del DataFrame original de 'data_sayer' sin valores atipicos
data_sayer = data_sayer_clean

In [ ]:
# Eliminacion de las variables temporales utilizadas para los valores atipicos
del sayer_total_atipicosBajos, sayer_laghoras_atipicosBajos 
del sayer_laghoras_atipicosAltos, valores_atipicos, data_sayer_clean

## Eliminar registros duplicados.

In [ ]:
# Cantidad de elementos en el df
data_sayer.index.size

In [ ]:
# Cantidad de elementos en el df si se eliminan las filas con informacion identica
data_sayer.drop_duplicates().index.size

Haciendo una resta simple, podemos calcular que actualmente existen $5787-5520 = 267$ duplicados. Esto añade un gran sesgo al análisis posterior, por lo que se procede a eliminarlos.

In [ ]:
# Eliminacion de filas con informacion identica
data_sayer.drop_duplicates(inplace=True)

# Resetear el indice del DataFrame
data_sayer.reset_index(drop=True, inplace=True)

In [ ]:
# Informacion del df despues de eliminar filas con informacion identica
data_sayer.info()

# **Construcción y transformación de datos**

## Agregación y agrupamiento.
 Construir nuevas características o variables con las tecnicas descritas

Primero, vamos a agrupar los valores de la columna 'COMPLAINT' para separar los siguientes tipos de
mantenimiento:
* PREVENTIVO
* CORRECTIVO
* OTROS

In [ ]:
# Analizar los jobcodes. imprimiendo todos los unicos
data_sayer['Jobcode'].value_counts()

In [ ]:
JobTypePreventive = ['000041 - Servicio Preventivo A - LT',
                     '000058 - Preventivo Llantas', '000005 - Servicio Preventivo Caja Seca A1', 
                     '000042 - Servicio Preventivo B - LT', '000007 - Servicio Preventivo Dolly A1',
                     '000043 - Servicio Preventivo C - LT', '000073 - Derivado de preventivo Tracto',
                     '000074 - Derivado de preventivo Arrastre', '000001 - Servicio preventivo M1']

JobTypeCorrective = ['000013 - Correctivo Tractos', '000018 - Correctivo Dolly', '000061 - Daño Operativo',
                     '000014 - Correctivo Cajas', '000059 - Correctivo Llantas', '000062 - Daño Operativo Llanta']

JobTypeCorrective_Vial = ['000053 - Auxilio Carretero - Mecánico',
                          '000027 - Auxilio Carretero - Llantas', 
                          '000054 - Auxilio Carretero- Abastecimiento', ]

data_sayer['JobType'] = 'OTROS'

data_sayer.loc[data_sayer['Jobcode'].isin(JobTypePreventive), 'JobType'] = 'PREVENTIVO'
data_sayer.loc[data_sayer['Jobcode'].isin(JobTypeCorrective), 'JobType'] = 'CORRECTIVO'
data_sayer.loc[data_sayer['Jobcode'].isin(JobTypeCorrective_Vial), 'JobType'] = 'CORRECTIVO VIAL'

In [ ]:
#¿Qué porcentaje representan los mantenimiento preventivos segun el ComplaintType respecto a todas las operaciones de mantenimiento?
percent = (data_sayer.groupby('JobType')['UnitID'].count() / data_sayer.shape[0] * 100).round(2)
# Ordenar por porcentaje
percent = percent.sort_values(ascending=False)

# Imprimir los porcentajes de mantenimientos por tipo
for i in range(len(percent)):
    print(f" (Complaint Type) Los mantenimientos de tipo '{percent.index[i]}' representan el {percent.iloc[i]}% de todas las operaciones de mantenimiento.")


Posteriormente, vamos a agrupar los valores de la columna 'RepReason' para identificar los tipos de
mantenimiento:
* PREVENTIVO
* CORRECTIVO
* OTROS

In [ ]:
# Eliminacion de las variables temporales utilizadas para categorizar los tipos de reparaciones
del JobTypePreventive, JobTypeCorrective, JobTypeCorrective_Vial

In [ ]:
# Crear una nueva columna 'RepReasonCategory' basada en 'ComplaintType'
data_sayer['JobTypeSummary'] = data_sayer['JobType'].apply(
    lambda x: 'CORRECTIVO' if x.startswith('CORRECTIVO') else (
        'PREVENTIVO' if x.startswith('PREVENTIVO') else x)
)

## Reordenamiento de Columnas

In [ ]:
# Reordenar las columnas del DataFrame
data_sayer = data_sayer[['OrderID', 'UnitID', 'UnitType', 'UnitYear', 'kMS', 'OpenedDate', 'lagdias', 
                         'laghoras', 'Jobcode', 'JobType', 'JobTypeSummary', 'RepReason', 'COMPLAINT', 
                         'DESCRIPTION', 'TipoLinea', 'NumParte', 'TOTAL', 'CITA']]

# Reparando data_sayer (data_sayer2)

Debido a que tenemos multiples reparaciones por dia (En una misma entrada a taller se registran varias reparaciones) las frecuencias de reparacion no eran adecuadas (estaban sesgadas). 

Por ello, debemos agrupar todas las reparaciones correctivas / preventivas realizadas el mismo dia, para garantizar que solo haya un registro diario de reparaciones, con un costo adecuado y así poder determinar las frecuencias de forma adecuada

## Funcion Cuatrimestres

In [ ]:
# Crear columna 'OpenedTrimester' para clasificar los cuatrimestres en los que se abrieron las reparaciones
# Definir la función para asignar cuatrimestres
def get_cuatrimestre(month):
    if month in [1, 2, 3, 4]:
        return 1
    elif month in [5, 6, 7, 8]:
        return 2
    elif month in [9, 10, 11, 12]:
        return 3

In [ ]:
# Agrupar el df 'data_sayer' por UnitID, OpenedDate y ComplaintType, sumando los costos
data_sayer2 = data_sayer.groupby(['UnitID', 'OpenedDate', 'JobType'])['TOTAL'].sum().reset_index()

# Añadir ComplaintTypeSummary a 'data_sayer2'
data_sayer2['JobTypeSummary'] = data_sayer.groupby(['UnitID', 'OpenedDate', 'JobType'])['JobTypeSummary'].first().reset_index()['JobTypeSummary'].values

# Añadir lagdias a 'data_sayer2'
data_sayer2['lagdias'] = data_sayer.groupby(['UnitID', 'OpenedDate', 'JobType'])['lagdias'].mean().reset_index()['lagdias'].values

# Añadir laghoras a 'data_sayer2'
data_sayer2['laghoras'] = data_sayer.groupby(['UnitID', 'OpenedDate', 'JobType'])['laghoras'].mean().reset_index()['laghoras'].values

# Añadir UnitType a 'data_sayer2'
data_sayer2['UnitType'] = data_sayer.groupby(['UnitID', 'OpenedDate', 'JobType'])['UnitType'].first().reset_index()['UnitType'].values

# Añadir kms a 'data_sayer2'
data_sayer2['kMS'] = data_sayer.groupby(['UnitID', 'OpenedDate', 'JobType'])['kMS'].first().reset_index()['kMS'].values

# Añadir UnitYear a 'data_sayer2'
data_sayer2['UnitYear'] = data_sayer.groupby(['UnitID', 'OpenedDate', 'JobType'])['UnitYear'].first().reset_index()['UnitYear'].values

# Crear columna 'OpenedTrimester' para clasificar los trimestres de apertura
data_sayer2['OpenedTrimester'] = data_sayer2['OpenedDate'].dt.month.apply(get_cuatrimestre)

# Crear columna 'ClosedMonth' para identificar el mes de cierre de la reparacion}
# Se calcula mediante la sumna de OpenedDate y lagdias
data_sayer2['ClosedMonth'] = (data_sayer2['OpenedDate'] + pd.to_timedelta(data_sayer2['lagdias'], unit='d')).dt.month

# Creacion de una columna 'MaintenanceYear' para identificar el año de mantenimiento
data_sayer2['MaintenanceYear'] = data_sayer2['OpenedDate'].dt.year

# Reordenar las columnas como en data_sayer
data_sayer2 = data_sayer2[['UnitID', 'UnitType', 'UnitYear', 'kMS', 'OpenedDate','lagdias',
                           'laghoras', 'JobType', 'JobTypeSummary', 'TOTAL', 'OpenedTrimester', 'ClosedMonth', 'MaintenanceYear']]

## Funciones MTBF

In [ ]:
# Función para calcular el promedio de días entre reparaciones
def meanBtwnRepairs(dates):
    if len(dates) > 1:
        dates = sorted(dates)  # Ordenar las fechas
        # Calcular diferencias entre fechas consecutivas y regresar el promedio en días
        return pd.Series(dates).diff().mean().days
    else:
        return None  # Si solo hay una fecha


# Función para calcular el promedio de días entre reparaciones
def meanBtwnRepairsHours(dates):
    if len(dates) > 1:
        dates = sorted(dates)  # Ordenar las fechas
        # Calcular diferencias entre fechas consecutivas y regresar el promedio en horas
        return pd.Series(dates).diff().mean().total_seconds() / 3600
    else:
        return None  # Si solo hay una fecha

## sayer_maint2

In [ ]:
# Creacion de un DataFrame para el registro de reparaciones y fechas por cada unidad de la flota
# de Sayer Full

# Se agrupara la informacion por 'UnitID', 'UnitType'
# Se agrega la columna 'RepairCount' que obtiene la cantidad de reparaciones de una misma unidad
# Se agrega la columna 'RepairDates' que obtiene las fechas de las reparaciones de una misma unidad
sayer_maint2_byUnit = data_sayer2.groupby(['UnitID', 'UnitType']).agg(
    RepairCount=('OpenedDate', 'size'),
    RepairDates=('OpenedDate', lambda x: list(
        x + pd.to_timedelta(data_sayer2.loc[x.index, 'laghoras'], unit='h')))
).reset_index()


# Asegurarse de que las fechas estén en formato datetime y manejar errores
sayer_maint2_byUnit['RepairDates'] = sayer_maint2_byUnit['RepairDates'].apply(
    lambda x: pd.to_datetime(x))

## sayer_maint2_RepReason

In [ ]:
# Creacion de un DataFrame para el registro de reparaciones y fechas por cada unidad de la flota
# de Sayer Full por tipo de reparacion

# Se agrupara la informacion por 'UnitID' y UnitType
# Se agrega la columna 'RepairCount' que obtiene la cantidad de reparaciones de una misma unidad
# Se agrega la columna 'RepairDates' que obtiene las fechas de las reparaciones de una misma unidad
sayer_maint2_byUnit_byRepReason = data_sayer2.groupby(['UnitID', 'UnitType', 'UnitYear', 'JobTypeSummary']).agg(
    RepairCount=('OpenedDate', 'size'),
    RepairDates=('OpenedDate', lambda x: list(
        x + pd.to_timedelta(data_sayer.loc[x.index, 'laghoras'], unit='h')))
).reset_index()

In [ ]:
# Crear una nueva columna para el promedio de días entre reparaciones
sayer_maint2_byUnit['AvgDaysBetweenRepairs'] = sayer_maint2_byUnit['RepairDates'].apply(
    meanBtwnRepairs)
# Crear una nueva columna para el promedio de horas entre reparaciones
sayer_maint2_byUnit['AvgHoursBetweenRepairs'] = sayer_maint2_byUnit['RepairDates'].apply(
    meanBtwnRepairsHours)

## sayer_maint2 corrective / preventive

In [ ]:
# Crear un nuevo DataFrame con las reparaciones correctivas
sayer_maint2_byUnit_corrective = sayer_maint2_byUnit_byRepReason[
    sayer_maint2_byUnit_byRepReason['JobTypeSummary'] == 'CORRECTIVO'].copy()

# Crear un nuevo DataFrame con las reparaciones preventivas
sayer_maint2_byUnit_preventive = sayer_maint2_byUnit_byRepReason[
    sayer_maint2_byUnit_byRepReason['JobTypeSummary'] == 'PREVENTIVO'].copy()

In [ ]:
# Borrado de la columna 'ComplaintType' del DataFrame 'sayer_maint2_byUnit_corrective' y 'sayer_maint2_byUnit_preventive'
sayer_maint2_byUnit_corrective.drop('JobTypeSummary', axis=1, inplace=True)
sayer_maint2_byUnit_preventive.drop('JobTypeSummary', axis=1, inplace=True)

# ==============================================

# Dataframes por año

In [ ]:
# Creacion de un DataFrame para los mantenimientos del año 2022
data_sayer2_2022 = data_sayer2[data_sayer2['MaintenanceYear'] == 2022]

# Creacion de un DataFrame para los mantenimientos del año 2023
data_sayer2_2023 = data_sayer2[data_sayer2['MaintenanceYear'] == 2023]

# Creacion de un DataFrame para los mantenimientos del año 2024
data_sayer2_2024 = data_sayer2[data_sayer2['MaintenanceYear'] == 2024]

In [ ]:
# Borrado de la columna 'MaintenanceYear' de los DataFrames creados, pues ya no es necesaria
data_sayer2_2022 = data_sayer2_2022.drop('MaintenanceYear', axis=1)
data_sayer2_2023 = data_sayer2_2023.drop('MaintenanceYear', axis=1)
data_sayer2_2024 = data_sayer2_2024.drop('MaintenanceYear', axis=1)

# Estadística hasta el momento

## Antiguedad de la flota

In [ ]:
# Promedio de edad por cada Unidad segun su UnitID
print(f'El promedio de edad de las unidades de tipo DOLLY es de: ', end='')
print(f"{2024 - data_sayer2[data_sayer2['UnitType'] == 'DOLLY'].groupby('UnitID')['UnitYear'].mean().mean()} años")
print(f'El promedio de edad de las unidades de tipo TRACTOR es de: ', end='')
print(f"{2024 - data_sayer2[data_sayer2['UnitType'] == 'TRACTOR'].groupby('UnitID')['UnitYear'].mean().mean()} años")
print(f'El promedio de edad de las unidades de tipo TRAILER es de: ', end='')
print(f"{2024 - data_sayer2[data_sayer2['UnitType'] == 'TRAILER'].groupby('UnitID')['UnitYear'].mean().mean():.2f} años")

## Muestras

In [ ]:
# Cantidad de elementos en la poblablacion (data)
print(f'Nuestra poblacion (toda la base de datos) es de \033[4m{data.index.size}\033[0m elementos')
# Cantidad de elementos en la muestra (data_sayer)
print(f'Nuestra muestra (proyecto Sayer Full) es de \033[4m{data_sayer.index.size}\033[0m elementos')
# Cantida de elementos en la muestra (data_sayer2)
print(f'Nuestra muestra (proyecto Sayer Full) es de \033[4m{data_sayer2.index.size}\033[0m elementos')
# Cantidad de elementos en la muestra (data_sayer_2022)
print(f'Nuestra muestra (Sayer 2022) es de \033[4m{data_sayer2_2022.index.size}\033[0m elementos')
# Cantidad de elementos en la muestra (data_sayer_2023)
print(f'Nuestra muestra (Sayer 2023) es de \033[4m{data_sayer2_2023.index.size}\033[0m elementos')
# Cantidad de elementos en la muestra (data_sayer_2024)
print(f'Nuestra muestra (Sayer 2024) es de \033[4m{data_sayer2_2024.index.size}\033[0m elementos')

In [ ]:
# Tipos de unidades en la muestra (data_sayer). Agrupar por tipo de unidad y contar cuantas distintas hay segun su UnitID
print(f'\nEn  Sayer Full, se cuenta con:')
data_sayer2.groupby("UnitType")["UnitID"].nunique().reset_index().rename(columns={"UnitID": "Unidades"})

## Reparaciones

### MTBR

In [ ]:
# Cantidad de reparaciones por unidad de la flota de Sayer Full
print(f'\nEn Sayer Full, se tiene la siguiente \033[4m\033[1mfrecuencia de reparaciones:\033[0m')
sayer_maint2_byUnit[['UnitID', 'UnitType','RepairCount', 'AvgDaysBetweenRepairs', 'AvgHoursBetweenRepairs']]

#### MTBR por UnitType

In [ ]:
# Tiempo promedio entre reparaciones por 'UnitType' en la flota de Sayer Full
print(f'\nEn Sayer Full, se tiene el siguiente \033[4m\033[1mpromedio de tiempo entre reparaciones (hrs):\033[0m\n')
sayer_maint2_byUnit.groupby('UnitType')['AvgHoursBetweenRepairs'].mean().reset_index()

#### MTTR por ComplaintType

In [ ]:
# Tiempo promedio en reparacion segun el tipo de reparacion (ComplaintType)
print(f'\nEn Sayer Full, se tiene el siguiente \033[4m\033[1mpromedio de tiempo en reparacion (MTTR):\033[0m\n')
data_sayer2.groupby('JobTypeSummary')['laghoras'].mean().reset_index()

### Distribucion de reparaciones

In [ ]:
# Cantidad de reparaciones segun el tipo de reparacion (ComplaintType)
print(f'\nEn Sayer Full, se tiene la siguiente \033[4m\033[1mdistribucion de reparaciones:\033[0m\n')
data_sayer2["JobTypeSummary"].value_counts().reset_index().rename(columns={"count": "Frecuencia"})

In [ ]:
# Cantidad de reparaciones segun el tipo de reparacion (ComplaintTypeSummary)
print(f'\nEn Sayer Full, se tiene la siguiente \033[4m\033[1mdistribucion de reparaciones:\033[0m\n')
data_sayer2["JobTypeSummary"].value_counts().reset_index().rename(columns={"count": "Frecuencia"})

In [ ]:
data_sayer2["MaintenanceYear"].value_counts().reset_index().rename(columns={"count": "Frecuencia"})

In [ ]:
data_sayer2.groupby(['MaintenanceYear', 'JobType']).size().reset_index().rename(columns={0: 'Frecuencia'})

In [ ]:
# Reparaciones por año por unitID
print(f'\nEn Sayer Full, se tiene la siguiente \033[4m\033[1mdistribucion de reparaciones por año:\033[0m\n')
z_temp = data_sayer2.groupby(['MaintenanceYear', 'UnitID']).size().reset_index().rename(columns={0: 'Frecuencia'})

In [ ]:
z_temp.groupby('MaintenanceYear')['Frecuencia'].mean().reset_index()

#### Repairs per Month

In [ ]:
# Graficar la cantidad de reparaciones por mes para los distintos años

# Cantidad de graficas a mostrar
figure, axis = plt.subplots(1, 3)
# Tamaño de las graficas
figure.set_size_inches(16, 6)
# Grafica 2022
sns.countplot(data=data_sayer2_2022, ax=axis[0], x='ClosedMonth', color='dimgray')
axis[0].set_title('Cantidad de reparaciones por mes en 2022')
axis[0].set_xlabel('Mes')
axis[0].set_ylabel('Cantidad de reparaciones')
# Grafica 2023
sns.countplot(data=data_sayer2_2023, ax=axis[1], x='ClosedMonth', color='dimgray')
axis[1].set_title('Cantidad de reparaciones por mes en 2023')
axis[1].set_xlabel('Mes')
axis[1].set_ylabel('Cantidad de reparaciones')
# Grafica 2024
sns.countplot(data=data_sayer2_2024, ax=axis[2], x='ClosedMonth', color='dimgray')
axis[2].set_title('Cantidad de reparaciones por mes en 2024')
axis[2].set_xlabel('Mes')
axis[2].set_ylabel('Cantidad de reparaciones')

# Añadir el conteo de reparaciones por tipo (ComplaintType) en cada mes
sns.countplot(data=data_sayer2_2022, ax=axis[0], x='ClosedMonth', hue='JobTypeSummary', hue_order=['CORRECTIVO', 'PREVENTIVO', 'OTROS'], palette='Accent_r')
sns.countplot(data=data_sayer2_2023, ax=axis[1], x='ClosedMonth', hue='JobTypeSummary', hue_order=['CORRECTIVO', 'PREVENTIVO', 'OTROS'], palette='Accent_r')
sns.countplot(data=data_sayer2_2024, ax=axis[2], x='ClosedMonth', hue='JobTypeSummary', hue_order=['CORRECTIVO', 'PREVENTIVO', 'OTROS'], palette='Accent_r')

plt.show()

#### Repairs per Cuatrimester

In [ ]:
# Graficar la cantidad de reparaciones por Cuatrimestre para los distintos años

# Cantidad de graficas a mostrar
figure, axis = plt.subplots(1, 3)

# Tamaño de las graficas
figure.set_size_inches(16, 6)

# Grafica 2022
sns.countplot(data=data_sayer2_2022, ax=axis[0], x='OpenedTrimester', color='dimgray')
axis[0].set_title('Cantidad de reparaciones por cuatrimestre en 2022')
axis[0].set_xlabel('Cuatrimestre')
axis[0].set_ylabel('Cantidad de reparaciones')

# Grafica 2023
sns.countplot(data=data_sayer2_2023, ax=axis[1], x='OpenedTrimester', color='dimgray')
axis[1].set_title('Cantidad de reparaciones por cuatrimestre en 2023')
axis[1].set_xlabel('Cuatrimestre')
axis[1].set_ylabel('Cantidad de reparaciones')

# Grafica 2024
sns.countplot(data=data_sayer2_2024, ax=axis[2], x='OpenedTrimester', color='dimgray')
axis[2].set_title('Cantidad de reparaciones por cuatrimestre en 2024')
axis[2].set_xlabel('Cuatrimestre')
axis[2].set_ylabel('Cantidad de reparaciones')

# Añadir el conteo de reparaciones por tipo (ComplaintType) en cada trimestre

sns.countplot(data=data_sayer2_2022, ax=axis[0], x='OpenedTrimester', hue='JobTypeSummary', hue_order=['CORRECTIVO', 'PREVENTIVO', 'OTROS'], palette='Accent_r')
sns.countplot(data=data_sayer2_2023, ax=axis[1], x='OpenedTrimester', hue='JobTypeSummary', hue_order=['CORRECTIVO', 'PREVENTIVO', 'OTROS'], palette='Accent_r')
sns.countplot(data=data_sayer2_2024, ax=axis[2], x='OpenedTrimester', hue='JobTypeSummary', hue_order=['CORRECTIVO', 'PREVENTIVO', 'OTROS'], palette='Accent_r')

plt.show()

In [ ]:
# Eliminacion de las variables temporales
del figure, axis

## Costos

### por UnitID

#### Totales

In [ ]:
# Calcular la suma del costo total de las reparaciones para cada unidad de la flota
print(f'Costo total de las reparaciones por unidad de la flota de Sayer Full:')
data_sayer2.groupby('UnitID')['TOTAL'].sum().reset_index().sort_values('TOTAL', ascending=False)

#### Promedios

In [ ]:
# Calcular la suma del costo promedio de las reparaciones para cada unidad de la flota
print(f'Costo promedio de las reparaciones por unidad de la flota de Sayer Full:')
data_sayer2.groupby('UnitID')['TOTAL'].mean().reset_index().sort_values('UnitID', ascending=False)

In [ ]:
# Calcular la suma del costo promedio de las reparaciones para cada unidad de la flota por tipo de reparacion
print(f'Costo promedio de las reparaciones por unidad de la flota de Sayer Full por tipo de reparacion:')
z_temp = data_sayer2.groupby(['UnitID', 'JobTypeSummary'])['TOTAL'].mean().reset_index().sort_values('UnitID', ascending=False)

### por UnitType

#### Totales

In [ ]:
# Costo total por tipo de unidad
print(f'Costo total de las reparaciones por tipo de unidad:')
data_sayer2.groupby('UnitType')['TOTAL'].sum().reset_index()

#### Promedios

In [ ]:
# Costo promedio por tipo de unidad
print(f'Costo promedio de las reparaciones por tipo de unidad:')
data_sayer2.groupby('UnitType')['TOTAL'].mean().reset_index()

### por UnitYear

#### Totales

In [ ]:
# Costo total por año de unidad
print(f'Costo total de las reparaciones por tipo de unidad:')
data_sayer2.groupby(['UnitYear', 'UnitType'])['TOTAL'].sum().reset_index()

#### Promedios

In [ ]:
# Costo promedio por año de unidad
print(f'Costo promedio de las reparaciones por tipo de unidad:')
data_sayer2.groupby(['UnitYear', 'UnitType'])['TOTAL'].mean().reset_index()

### por ComplaintType

#### Totales

In [ ]:
# Costo total por tipo de reparacion (ComplaintType)
print(f'Costo total de las reparaciones por tipo de reparacion:')
data_sayer2.groupby('JobType')['TOTAL'].sum().reset_index()

#### Promedios

In [ ]:
# Costo promedio por tipo de reparacion (ComplaintType)
print(f'Costo promedio de las reparaciones por tipo de reparacion:')
data_sayer2.groupby('JobType')['TOTAL'].mean().reset_index()

### por Año

#### Totales

In [ ]:
# Costos totales por año de reparacion
print(f'Costo total de las reparaciones por año:')
data_sayer2.groupby(['MaintenanceYear'])['TOTAL'].sum().reset_index()

In [ ]:
# Costos totales por año y cuatrimestre de reparacion
print(f'Costo total de las reparaciones por año y trimestre:')
data_sayer2.groupby(['MaintenanceYear', 'OpenedTrimester'])['TOTAL'].sum().reset_index()

In [ ]:
# Costos totales por año de reparacion y tipo de reparacion
print(f'Costo totaL de las reparaciones por año & ComplaintTypeSummary:')
data_sayer2.groupby(['MaintenanceYear', 'JobTypeSummary'])['TOTAL'].sum().reset_index()

In [ ]:
# Costos totales por año de reparacion
print(f'Costo total de las reparaciones por año & ComplaintType:')
data_sayer2.groupby(['MaintenanceYear', 'JobType'])['TOTAL'].sum().reset_index()

# Pruebas de Hipótesis

## Reparaciones

Por: UnitYear, UnitType, ComplaintType

In [ ]:
# Conteo de las reparaciones por año, tipo de unidad y tipo de reparacion a traves del conteo de registros
data_sayer2.groupby(['UnitYear', 'UnitType', 'JobType'])['UnitID'].count().reset_index().rename(columns={"UnitID": "Frecuency"})

In [ ]:
# Guardar el agrupamiento en un nuevo DataFrame
sayer2_analysis = data_sayer2.groupby(['UnitYear', 'UnitType', 'JobType'])['UnitID'].count().reset_index()

# Renombrar la columna 'UnitID' por 'RepairCount'
sayer2_analysis.rename(columns={'UnitID': 'RepairCount'}, inplace=True)

# Porcentaje de reparaciones correctivas sobre el total de reparaciones por año y tipo de unidad
sayer2_analysis['PercentOverTotal'] = sayer2_analysis['RepairCount'] / sayer2_analysis.groupby(['UnitYear', 'UnitType'])['RepairCount'].transform('sum')

print(f'Porcentaje de reparaciones correctivas de Cajas de 2016:')
print(f'{sayer2_analysis.iloc[0, 4] * 100:.2f}%')

print(f'Porcentaje de reparaciones correctivas de Cajas de 2017:')
print(f'{sayer2_analysis.iloc[3, 4] * 100:.2f}%')

print(f'Porcentaje de reparaciones correctivas de Cajas de 2018:')
print(f'{sayer2_analysis.iloc[6, 4] * 100:.2f}%')

In [ ]:
# Frencuencia de reparaciones por año y tipo de unidad
print(f'Frecuencia de reparaciones por año y tipo de unidad:')
sayer2_analysis.groupby(['UnitYear', 'UnitType'])['RepairCount'].sum().reset_index().rename(columns={"RepairCount": "Frequency"})

## MTTR

In [ ]:
# laghoras promedio por tipo de reparacion (ComplaintType)
print(f'Promedio de horas por tipo de reparacion:')
print(f"\n{data_sayer2.groupby('JobType')['laghoras'].mean().reset_index().rename(columns={'laghoras': 'MTTR'})}")

print(f"\n{data_sayer2.groupby('JobTypeSummary')['laghoras'].mean().reset_index().rename(columns={'laghoras': 'MTTR'})}")

## Distribucion de los datos

### MTTR

In [ ]:
# Crear un histograma de la distribución de las horas promedio entre reparaciones
plt.figure(figsize=(16, 8))
sns.histplot(data_sayer2['laghoras'], kde=True)
plt.title('Distribución de las "horas promedio" (MTTR) en reparaciones. RAW')
plt.xlabel('Horas')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Aplicar una transformacion logaritmica
scaler = PowerTransformer(method='yeo-johnson')
pow_horas = scaler.fit_transform(data_sayer2['laghoras'].values.reshape(-1, 1))

In [ ]:
# Crear un grafico QQ-Plot (Cuantil-Cuantil) para verificar la normalidad de los datos
stats.probplot(pow_horas.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Horas de reparacion" (MTTR)')
plt.show()

In [ ]:
# Crear un histograma de la distribución de las horas promedio entre reparaciones
plt.figure(figsize=(16, 8))
sns.histplot(pow_horas, kde=True)
plt.title('Distribución de las "horas promedio" (MTTR) en reparaciones. Normalizado')
plt.xlabel('Horas')
plt.ylabel('Frecuencia')
plt.show()

### Costos

In [ ]:
# Histograma de la distribución de los costos de mantenimiento
plt.figure(figsize=(16, 8))
sns.histplot(data_sayer2['TOTAL'], kde=True)
plt.title('Distribución de los costos de mantenimiento. RAW')
plt.xlabel('Costo')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Aplicar una transformacion logaritmica
scaler = PowerTransformer(method='yeo-johnson')
pow_total = scaler.fit_transform(data_sayer2['TOTAL'].values.reshape(-1, 1))

In [ ]:
# Asignar la transformacion a una nueva columna
data_sayer2['TOTAL_norm'] = pow_total

In [ ]:
# Crear un grafico QQ-Plot (Cuantil-Cuantil) para verificar la normalidad de los datos
stats.probplot(pow_total.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Costos de mantenimiento"')
plt.show()

In [ ]:
# Histograma de la distribución de los costos de mantenimiento
plt.figure(figsize=(16, 8))
sns.histplot(pow_total, kde=True)
plt.title('Distribución de los costos de mantenimiento. Normalizado')
plt.xlabel('Costo')
plt.ylabel('Frecuencia')
plt.show()

### Mantenimiento Correctivo

In [ ]:
# Frecuencias de mantenimiento correctivo por tipo de unidad (UnitType)
print(f'Frecuencia de mantenimiento correctivo por tipo de unidad:')
sayer_maint2_byUnit_corrective.groupby('UnitType')['RepairCount'].sum().reset_index().rename(columns={"RepairCount": "Frequency"})

In [ ]:
# Histograma de las frecuencias de mantenimiento correctivo
plt.figure(figsize=(16, 8))
sns.histplot(sayer_maint2_byUnit_corrective['RepairCount'], kde=True)
plt.title('Distribución de las frecuencias de mantenimiento correctivo. RAW')
plt.xlabel('Frecuencia de mantenimiento correctivo')
plt.ylabel('Cantidad de unidades')

In [ ]:
# Aplicar una transformacion logaritmica
scaler = PowerTransformer(method='yeo-johnson')
pow_repaircount = scaler.fit_transform(sayer_maint2_byUnit_corrective['RepairCount'].values.reshape(-1, 1))

In [ ]:
# Añaadir la transformacion a una nueva columna
sayer_maint2_byUnit_corrective['RepairCount_norm'] = pow_repaircount

In [ ]:
# Creacion de un grafico QQ-Plot (Cuantil-Cuantil) para verificar la normalidad de los datos
stats.probplot(pow_repaircount.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Frecuencia de mantenimiento correctivo"')
plt.show()

In [ ]:
# Histograma de las frecuencias de mantenimiento correctivo
plt.figure(figsize=(16, 8))
sns.histplot(pow_repaircount, kde=True)
plt.title('Distribución de las frecuencias de mantenimiento correctivo. Normalizado')
plt.xlabel('Frecuencia de mantenimiento correctivo.')
plt.ylabel('Cantidad de unidades')

### Mantenimiento Preventivo

In [ ]:
# Frecuencias de mantenimiento correctivo por tipo de unidad (UnitType)
print(f'Frecuencia de mantenimiento preventivo por tipo de unidad:')
sayer_maint2_byUnit_preventive.groupby('UnitType')['RepairCount'].sum().reset_index().rename(columns={"RepairCount": "Frequency"})

In [ ]:
# Histograma de las frecuencias de mantenimiento correctivo
plt.figure(figsize=(16, 8))
sns.histplot(sayer_maint2_byUnit_preventive['RepairCount'], kde=True)
plt.title('Distribución de las frecuencias de mantenimiento preventivo. RAW')
plt.xlabel('Frecuencia de mantenimiento preventivo')
plt.ylabel('Cantidad de unidades')

In [ ]:
# Aplicar una transformacion logaritmica
scaler = PowerTransformer(method='yeo-johnson')
pow_repaircount = scaler.fit_transform(sayer_maint2_byUnit_preventive['RepairCount'].values.reshape(-1, 1))

In [ ]:
# Añadir la transformacion a una nueva columna
sayer_maint2_byUnit_preventive['RepairCount_norm'] = pow_repaircount

In [ ]:
# Creacion de un grafico QQ-Plot (Cuantil-Cuantil) para verificar la normalidad de los datos
stats.probplot(pow_repaircount.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Frecuencia de mantenimiento preventivo"')
plt.show()

In [ ]:
# Histograma de las frecuencias de mantenimiento correctivo
plt.figure(figsize=(16, 8))
sns.histplot(pow_repaircount, kde=True)
plt.title('Distribución de las frecuencias de mantenimiento preventivo. Normalizado')
plt.xlabel('Frecuencia de mantenimiento preventivo.')
plt.ylabel('Cantidad de unidades')

In [ ]:
# Eliminacion de las variables temporales utilizadas para la transformacion de las variables
del pow_horas, pow_total, pow_repaircount, scaler

# Hand's On 3A

## Correlaciones

### Preventive Freq. - Maint. Costs

In [ ]:
# Registros de data_sayer con ComplaintType 'PREVENTIVE'. Obtener el promedio de sus costos
sayer_maint2_byUnit_preventive['Total'] = data_sayer2[data_sayer2['JobTypeSummary'] == 'PREVENTIVO'].groupby('UnitID')['TOTAL'].sum().values

In [ ]:
# Sacar la correlacion entre las variables 'RepaitCount' y 'TotalCost' RAW
sayer_maint2_byUnit_preventive[['RepairCount', 'Total']].corr()

In [ ]:
# Grafica QQ-Plot para verificar la normalidad de los datos
stats.probplot(sayer_maint2_byUnit_preventive['Total'], dist="norm", plot=plt)
plt.title('QQ-Plot "Costos de mantenimiento preventivo"')
plt.show()

In [ ]:
# Normalizar los datos de 'MeanCost'
scaler = PowerTransformer(method='yeo-johnson')
pow_meancost = scaler.fit_transform(sayer_maint2_byUnit_preventive['Total'].values.reshape(-1, 1))

In [ ]:
# Grafica QQ-Plot para verificar la normalidad de los datos
stats.probplot(pow_meancost.flatten(), dist="norm", plot=plt)
plt.title('QQ-Plot "Costos de mantenimiento preventivo"')
plt.show()

In [ ]:
# Añadir la transformacion a una nueva columna
sayer_maint2_byUnit_preventive['Total_norm'] = pow_meancost

In [ ]:
# Sacar la correlacion entre las variables 'RepaitCount' y 'TotalCost' normalizadas
sayer_maint2_byUnit_preventive[['RepairCount_norm', 'Total_norm']].corr()

Como podemos observar, la relacion entre las columnas sin normalizar es mejor que las columnas normalizadas

In [ ]:
# Scatter plot between 'RepairCount' and 'Total'
plt.figure(figsize=(16, 8))
sns.scatterplot(x='RepairCount', y='Total', data=sayer_maint2_byUnit_preventive)
plt.title('Correlación entre Frecuencia y Costo de mantenimiento preventivo')
plt.xlabel('Frecuencia de mantenimiento preventivo')
plt.ylabel('Costo de mantenimiento preventivo')
plt.show()

#### Modelo de Regresión Lineal

In [ ]:
# Preparar el dataframe necesario para el modelo de regresion
# Debe incluir la frecuencia de Reparaciones por cada Unidad, Año, Cuatrimestre y Tipo de Reparacion
modelA = data_sayer2.groupby(['UnitID', 'MaintenanceYear', 'OpenedTrimester', 'JobTypeSummary'])['OpenedDate'].count().reset_index()

#Renombrar la columna 'OpenedDate' por 'RepairCount'
modelA.rename(columns={'OpenedDate': 'RepairCount'}, inplace=True)

# Añadir una columna 'Total' que contenga el costo total de las reparaciones
modelA['Total'] = data_sayer2.groupby(['UnitID', 'MaintenanceYear', 'OpenedTrimester', 'JobTypeSummary'])['TOTAL'].sum().values

# Obtener el dataframe para reparaciones preventivas
modelA_preventive = modelA[modelA['JobTypeSummary'] == 'PREVENTIVO'].copy()

In [ ]:
# Crear un modelo de regresion lineal, con las variables 'RepairCount' y 'Total'

X = modelA_preventive[['RepairCount']]  # Variable independiente
y = modelA_preventive['Total']    # Variable dependiente

In [ ]:
# Dividir los datos en en train y test

# Datos de entrenamiento (Todos los de cuatrimestres 1 y 2 de 2022 y 2023)
X_train = modelA_preventive[(modelA_preventive['MaintenanceYear'] != 2024) & (modelA_preventive['OpenedTrimester'] != 3)][['RepairCount']]
y_train = modelA_preventive[(modelA_preventive['MaintenanceYear'] != 2024) & (modelA_preventive['OpenedTrimester'] != 3)]['Total']

# Datos de prueba (Todos los de cuatrimestres 3 de 2022 y 2023)
X_test = modelA_preventive[(modelA_preventive['MaintenanceYear'] != 2024) & (modelA_preventive['OpenedTrimester'] == 3)][['RepairCount']]
y_test = modelA_preventive[(modelA_preventive['MaintenanceYear'] != 2024) & (modelA_preventive['OpenedTrimester'] == 3)]['Total']

# Creación del modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X=X_train, y=y_train)

In [ ]:
# Información del modelo
print(f"Intercept: {modelo.intercept_}")
print(f"Coeficiente: {modelo.coef_}")

In [ ]:
# Evaluación en el conjunto de entrenamiento
y_train_pred = modelo.predict(X_train)
print(f"R^2 en train: {modelo.score(X_train, y_train)}")

In [ ]:
# Evaluación en el conjunto de prueba
y_test_pred = modelo.predict(X_test)
print(f"R^2 en test: {r2_score(y_test, y_test_pred)}")
print(f"RMSE en test: {np.sqrt(mean_squared_error(y_test, y_test_pred))}")

In [ ]:
# Crear el modelo con statsmodels para obtener un resumen detallado
X_train_sm = sm.add_constant(X_train)  # Agregar término de intersección
modelo_sm = sm.OLS(y_train, X_train_sm).fit()
print(modelo_sm.summary())

In [ ]:
# Intervalos de confianza para los coeficientes del modelo
intervalos_ci = modelo_sm.conf_int(alpha=0.05)
intervalos_ci.columns = ['2.5%', '97.5%']
intervalos_ci

In [ ]:
# Predicciones con intervalo de confianza del 95%
predicciones = modelo_sm.get_prediction(exog=X_train_sm).summary_frame(alpha=0.05)
predicciones['x'] = X_train_sm.loc[:, 'RepairCount']
predicciones['y'] = y_train
predicciones = predicciones.sort_values('x')

# Gráfico del modelo
fig, ax = plt.subplots(figsize=(6, 3.84))

ax.scatter(predicciones['x'], predicciones['y'], marker='o', color = "gray")
ax.plot(predicciones['x'], predicciones["mean"], linestyle='-', label="OLS")
ax.plot(predicciones['x'], predicciones["mean_ci_lower"], linestyle='--', color='red', label="95% CI")
ax.plot(predicciones['x'], predicciones["mean_ci_upper"], linestyle='--', color='red')
ax.fill_between(predicciones['x'], predicciones["mean_ci_lower"], predicciones["mean_ci_upper"], alpha=0.3)
ax.legend();

In [ ]:
# Error de test del modelo
X_test_sm = sm.add_constant(X_test, prepend=True)
predicciones = modelo_sm.predict(exog=X_test_sm)
rmse = root_mean_squared_error(y_true=y_test, y_pred=predicciones)
print(f"El error (rmse) de test es: {rmse}")

In [ ]:
# Eliminacion de las variables temporales
del modelA, modelA_preventive
del X, X_train, X_train_sm, X_test, y, y_train, y_train_pred, y_test, y_test_pred,
del modelo, modelo_sm
del ax, fig
del intervalos_ci, predicciones, rmse, X_test_sm

### Modelo de Regresión Múltiple

In [ ]:
modelB = data_sayer2[['UnitID', 'UnitYear', 'kMS', 'OpenedDate', 'JobTypeSummary', 'OpenedTrimester',
                     'MaintenanceYear', 'laghoras', 'TOTAL']].copy()

# actualizar el modelo para que solo contenga registros con kMS no nulos
modelB = modelB[~modelB['kMS'].isnull()]

# Convertir la columna ModelYear a la cantidad de años que tiene el vehiculo
modelB['UnitYear'] = 2024 - modelB['UnitYear']

# Renombrar la columna 'UnitYear' por 'UnitAge'
modelB.rename(columns={'UnitYear': 'UnitAge'}, inplace=True)

# Renombrar la columna 'ñaghoras' por 'MTTR'
modelB.rename(columns={'laghoras': 'MTTR'}, inplace=True)

In [ ]:
# Obtener el dataframe para reparaciones correctivas
modelB_corrective = modelB[modelB['JobTypeSummary'] == 'CORRECTIVO'].copy()

In [ ]:
# Crear un modelo de regresion lineal, con las variables 'UnitAge', 'kMS', 'MTTR' y 'Total'
X = modelB_corrective[['UnitAge', 'kMS', 'MTTR']]  # Variable independiente
y = modelB_corrective['TOTAL']    # Variable dependiente

In [ ]:
# Dividir los datos en en train y test

# Datos de entrenamiento (Todos los de cuatrimestres 1 y 2
X_train = modelB_corrective[modelB_corrective['OpenedTrimester'] != 3][['UnitAge', 'kMS', 'MTTR']]
y_train = modelB_corrective[modelB_corrective['OpenedTrimester'] != 3]['TOTAL']

# Datos de prueba (Todos los de cuatrimestre 3 de 2022 y 2023)
X_test = modelB_corrective[(modelB_corrective['MaintenanceYear'] != 2024) & (modelB_corrective['OpenedTrimester'] == 3)][['UnitAge', 'kMS', 'MTTR']]
y_test = modelB_corrective[(modelB_corrective['MaintenanceYear'] != 2024) & (modelB_corrective['OpenedTrimester'] == 3)]['TOTAL']

# Creación del modelo de regresión lineal multiple
modelo = LinearRegression()

# Ajustar el modelo
modelo.fit(X=X_train, y=y_train)

In [ ]:
# Información del modelo
print(f"Intercept: {modelo.intercept_}")
print(f"Coeficiente: {modelo.coef_}")

In [ ]:
# Evaluación en el conjunto de entrenamiento
y_train_pred = modelo.predict(X_train)
print(f"R^2 en train: {modelo.score(X_train, y_train)}")

In [ ]:
# Evaluación en el conjunto de prueba
y_test_pred = modelo.predict(X_test)
print(f"R^2 en test: {r2_score(y_test, y_test_pred)}")
print(f"RMSE en test: {np.sqrt(mean_squared_error(y_test, y_test_pred))}")

In [ ]:
# Crear el modelo con statsmodels para obtener un resumen detallado
X_train_sm = sm.add_constant(X_train)  # Agregar término de intersección
modelo_sm = sm.OLS(y_train, X_train_sm).fit()
print(modelo_sm.summary())

In [ ]:
# Intervalos de confianza para los coeficientes del modelo
intervalos_ci = modelo_sm.conf_int(alpha=0.05)
intervalos_ci.columns = ['2.5%', '97.5%']
intervalos_ci

In [ ]:
# Error de test del modelo
X_test_sm = sm.add_constant(X_test, prepend=True)
predicciones = modelo_sm.predict(exog=X_test_sm)
rmse = root_mean_squared_error(y_true=y_test, y_pred=predicciones)
print(f"El error (rmse) de test es: {rmse}")

In [ ]:
# Eliminacion de las variables temporales
del modelB, modelB_corrective
del X, X_train, X_train_sm, X_test, y, y_train, y_train_pred, y_test, y_test_pred,
del modelo, modelo_sm
del intervalos_ci, predicciones, rmse, X_test_sm

### Corrective Freq. - UnitYear

In [ ]:
# Sacar la correlacion entre las variables 'RepairCount' y 'UnitYear'
sayer_maint2_byUnit_corrective[['RepairCount', 'UnitYear']].corr()

In [ ]:
# Normalizar los datos de 'RepairCount'
scaler = PowerTransformer(method='yeo-johnson')
pow_repaircount = scaler.fit_transform(sayer_maint2_byUnit_corrective['RepairCount'].values.reshape(-1, 1))

In [ ]:
# Añadir la transformacion a una nueva columna
sayer_maint2_byUnit_corrective['RepairCount_norm'] = pow_repaircount

In [ ]:
# Sacar la correlacion entre las variables 'RepairCount' y 'UnitYear' normalizadas
sayer_maint2_byUnit_corrective[['RepairCount_norm', 'UnitYear']].corr()

Como podemos observar, la relacion entre las columnas normalizadas es mejor que las columnas naturales

### MaintType - UnitYear / kMS

In [ ]:
# Crear una columna en data_sayer2 para saber si la reparacion fue correctiva o preventiva
data_sayer2['MaintType'] = data_sayer2['JobType'].apply(lambda x: 1 if x == 'CORRECTIVO' else 0)

#### Modelo de Regresion Logística

In [ ]:
# Creacion de un DataFrame para el modelo de regresion
modelC = data_sayer2[['UnitID', 'UnitYear', 'kMS', 'laghoras', 'MaintType', 'TOTAL']].copy()

# Eliminar los registros con valores nulos en 'kMS'
modelC = modelC[~modelC['kMS'].isnull()]

In [ ]:
# Creacion de un Modelo de Regresion Logística para predecir si una reparacion es correctiva o preventiva
X = modelC[['kMS', 'UnitYear']]
y = modelC['MaintType']

In [ ]:
# Dividir los datos en en train y test
X_train, X_test, y_train, y_test = train_test_split(
                                                    X,
                                                    y,
                                                    train_size   = 0.75,
                                                    random_state = 42,
                                                    shuffle      = True
                                                    )

# Creación del modelo de regresión logística
modelo = LogisticRegression()
modelo.fit(X=X_train, y=y_train)

In [ ]:
# Evaluación en el conjunto de entrenamiento
y_train_pred = modelo.predict(X_train)
print(f"Accuracy en train: {accuracy_score(y_train, y_train_pred)}")

In [ ]:
# Evaluación en el conjunto de prueba
y_test_pred = modelo.predict(X_test)
print(f"Accuracy en test: {accuracy_score(y_test, y_test_pred)}")

In [ ]:
# Grafica ROC Curve para el modelo de regresion logistica
RocCurveDisplay.from_estimator(modelo, X_test, y_test)

# Obtener los valores de la curva ROC
fpr, tpr, _ = roc_curve(y_test, modelo.predict_proba(X_test)[:, 1])

# Sombrear el área bajo la curva ROC
plt.fill_between(fpr, tpr, alpha=0.3, color='gray')

# Añadir la pendiente y la intersección al gráfico
plt.plot([0, 1], [0, 1], linestyle='--', color='orange')
plt.xlim([0.0, 1.0])

# Añadir etiquetas y título
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Modelo de Regresion Logistica')

plt.show()

In [ ]:
# Eliminacion de las variables temporales
del modelC
del X, X_train, X_test, y, y_train, y_train_pred, y_test, y_test_pred
del modelo, fpr, tpr

# Convertir a HTML

In [ ]:
# %%shell
# jupyter nbconvert --to html /content/E5_Construcción_y_transformación_de_datos.ipynb